# Préparation des données pour la classification des vehicules des clients à l'aide des fichiers clients et des immatriculations

L'objectif est de : 
- Fusionner les données des clients avec ceux de l'immatriculation obtenu lors du clustering
- Préparer les données pour entrainer le modèle de recommandation de vehicules et faire une analyse exploratoire sur les données issus de la fusion entre les clients et les immatriculations

**Expliquer la méthodologie**

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
import matplotlib.cm as cm
import plotly.express as px
from scipy.spatial import ConvexHull
from matplotlib.lines import Line2D

### Chargement des données

In [3]:
# Données des clients
clients_51 = pd.read_csv("data/Clients_51.csv", encoding='latin-1')
clients_52 = pd.read_csv("data/Clients_52.csv", encoding='latin-1')
clients_5 = pd.concat([clients_51, clients_52])
clients_5.head(5)

,age,sexe,taux,situationFamiliale,nbEnfantsAcharge,2eme voiture,immatriculation
0,49,F,914,En Couple,1,false,2170 DJ 60
1,18,M,563,En Couple,4,false,8132 RT 49
2,82,M,417,Célibataire,0,false,4764 CE 84
3,72,M,442,En Couple,4,false,6239 YO 57
4,41,M,592,Célibataire,0,false,9318 FD 10


In [6]:
clients_5.shape

(200000, 7)

In [7]:
# Renommage de la colonne 2eme voiture
clients_5.rename(columns={"2eme voiture" : "2emevoiture"}, inplace=True)

In [8]:
# Données d'immatriculations
immatriculations:pd.DataFrame = None
with open("data_processed/clustering_immatriculations_categoriser_kmeans.pkl","rb") as f1:
    immatriculations = pickle.load(f1)
immatriculations.head(5)

,immatriculation,marque,nom,puissance,longueur,nbPlaces,nbPortes,couleur,occasion,prix,categorie_label,categorie
0,3176 TS 67,Renault,Laguna 2.0T,170,longue,5,5,blanc,False,27300,4,Moyen
1,3721 QS 49,Volvo,S80 T6,272,très longue,5,5,noir,False,50500,2,Familliale
2,9099 UV 26,Volkswagen,Golf 2.0 FSI,150,moyenne,5,5,gris,True,16029,3,Economique
3,3563 LA 55,Peugeot,1007 1.4,75,courte,5,5,blanc,True,9625,0,Citadine
4,6963 AX 34,Audi,A2 1.4,75,courte,5,5,gris,False,18310,3,Economique


### Préparation des données des clients

In [9]:
clients_data = clients_5.copy()

### Détection des valeurs manquantes

In [10]:
clients_data.isnull().sum()

age                   0
sexe                  0
taux                  0
situationFamiliale    0
nbEnfantsAcharge      0
2emevoiture           0
immatriculation       0
dtype: int64

Il n'y a pas de valeurs manquantes a priori

**Faire les liste des problèmes du jeu de données**. Faire un tableau résumant les problèmes | Méthodologie : Identifié les problèmes et les corriger

In [11]:
clients_data.dtypes

age                   object
sexe                  object
taux                  object
situationFamiliale    object
nbEnfantsAcharge      object
2emevoiture           object
immatriculation       object
dtype: object

* Age est de type object alors qu'il doit être de type numérique
* Taux est de type object alors qu'il doit être de type numérique
* Nombre d'enfant est de type object alors qu'il doit être de type numérique
* 2ème véhicule est de type object alors qu'il doit être de type booléen

Correction des types

In [12]:
clients_data.age = pd.to_numeric(clients_data.age, errors="coerce")
clients_data.taux = pd.to_numeric(clients_data.taux, errors="coerce")
clients_data.nbEnfantsAcharge = pd.to_numeric(clients_data.nbEnfantsAcharge, errors="coerce")

In [13]:
display(clients_data.dtypes)
print("_"*50)
display(clients_data.isnull().sum())
print("_"*50)
clients_data.isin([" ", "?", "N/D", "-1"]).sum()

age                   float64
sexe                   object
taux                  float64
situationFamiliale     object
nbEnfantsAcharge      float64
2emevoiture            object
immatriculation        object
dtype: object

__________________________________________________


age                   388
sexe                    0
taux                  417
situationFamiliale      0
nbEnfantsAcharge      401
2emevoiture             0
immatriculation         0
dtype: int64

__________________________________________________


age                     0
sexe                  586
taux                    0
situationFamiliale    632
nbEnfantsAcharge        0
2emevoiture           386
immatriculation         0
dtype: int64

On observe des valeurs, nulles, non attendues et des types incorrectes

**Remplacement des valeurs bizarres par Nan**

In [14]:
# Remplacement des valeurs inatendue par nan
clients_data.replace([" ", "?", "N/D", "-1",-1],np.nan,inplace=True)

In [15]:
clients_data.isnull().sum()

age                   569
sexe                  586
taux                  641
situationFamiliale    632
nbEnfantsAcharge      604
2emevoiture           386
immatriculation         0
dtype: int64

### Nettoyage des données

**Domaine de valeur incorrecte**

**Sexe**

In [16]:
clients_data.sexe.value_counts()

M           136690
F            58772
Masculin      1378
Homme         1332
Femme          651
Féminin        591
Name: sexe, dtype: int64

In [17]:
clients_data["sexe"] = sexe_vals = clients_data.sexe.map({
    "Masculin":"M",
    "Homme":"M",
    "Femme":"F",
    "Féminin":"F",
    "M":"M",
    "F":"F"
})
clients_data.sexe.value_counts()

M    139400
F     60014
Name: sexe, dtype: int64

**2emevoiture**

In [18]:
clients_data["2emevoiture"].value_counts()

false    173756
true      25858
Name: 2emevoiture, dtype: int64

In [19]:
clients_data["2emevoiture"] = clients_data["2emevoiture"].map({
    "true":True,
    "false":False
})
clients_data["2emevoiture"].value_counts()

False    173756
True      25858
Name: 2emevoiture, dtype: int64

**Situation familiale**

**Correction du domaine de valeurs**

In [20]:
clients_data.situationFamiliale.value_counts()

En Couple      128121
Célibataire     59415
Seule            9825
Marié(e)         1317
Seul              588
Divorcée          102
Name: situationFamiliale, dtype: int64

In [21]:
clients_data.situationFamiliale = clients_data.situationFamiliale.map({
    "En Couple": "En Couple", 
    "Célibataire": "Célibataire",
    "Seule" : "Célibataire", 
    "Marié(e)" : "En Couple", 
    "Seul" : "Célibataire",
    "Divorcée": "Célibataire"
})
clients_data.situationFamiliale.value_counts()

En Couple      129438
Célibataire     69930
Name: situationFamiliale, dtype: int64

###### **Bilan des valeurs manquantes**

Pourcentage des lignes manquantes.

In [22]:
(clients_data.isnull().sum() / clients_data.shape[0]) * 100

age                   0.2845
sexe                  0.2930
taux                  0.3205
situationFamiliale    0.3160
nbEnfantsAcharge      0.3020
2emevoiture           0.1930
immatriculation       0.0000
dtype: float64

In [23]:
# Nombre de lignes avec valeurs manquantes
nbr_miss_line = clients_data[clients_data.isna().any(axis=1)].shape[0]
nbr_miss_line

3396

In [24]:
x = (nbr_miss_line/clients_data.shape[0]) * 100
print("Le taux de lignes manquantes est :",x)

Le taux de lignes manquantes est : 1.698


La taux de valeurs manquantes est de 1.698 ce qui presente une faible partie de l'ensemble de données.

**Conclusion**

Le valeurs manquantes étant peu nombreuses par rapport à l'ensemble de données on peut se permettre de les supprimer. **(Préciser que le taux de valeur manquante est trop peu par rapport a l'ensemble de données, on peut se permettre de les supprimer)**

In [25]:
clients_data2 = clients_data.copy()
clients_data2.dropna(inplace=True)
clients_data2.isnull().sum()

age                   0
sexe                  0
taux                  0
situationFamiliale    0
nbEnfantsAcharge      0
2emevoiture           0
immatriculation       0
dtype: int64

**Ajustement des types**

In [26]:
clients_data2.dtypes

age                   float64
sexe                   object
taux                  float64
situationFamiliale     object
nbEnfantsAcharge      float64
2emevoiture            object
immatriculation        object
dtype: object

In [27]:
clients_data2["age"] = clients_data2["age"].astype(np.int64)
clients_data2["2emevoiture"] = clients_data2["2emevoiture"].astype(bool)
clients_data2["nbEnfantsAcharge"] = clients_data2["nbEnfantsAcharge"].astype(np.int64)
clients_data2["taux"] = clients_data2["taux"].astype(np.int64)

In [24]:
clients_data2.dtypes

age                    int64
sexe                  object
taux                   int64
situationFamiliale    object
nbEnfantsAcharge       int64
2emevoiture             bool
immatriculation       object
dtype: object

In [28]:
clients_data2.head(5)

,age,sexe,taux,situationFamiliale,nbEnfantsAcharge,2emevoiture,immatriculation
0,49,F,914,En Couple,1,False,2170 DJ 60
1,18,M,563,En Couple,4,False,8132 RT 49
2,82,M,417,Célibataire,0,False,4764 CE 84
3,72,M,442,En Couple,4,False,6239 YO 57
4,41,M,592,Célibataire,0,False,9318 FD 10


Les données sont maintenant propres.

**A partir d'ici les données sont propres on peu passer la suite**

### Fusion des données Client et Immatriculations

In [29]:
cols = ["age","sexe","taux","nbEnfantsAcharge","situationFamiliale","2emevoiture","categorie_label","categorie"]

In [30]:
clients_immatriculations = clients_data2.merge(immatriculations, on="immatriculation")

In [31]:
clients_immatriculations = clients_immatriculations[cols]

In [32]:
clients_immatriculations.head(10)

,age,sexe,taux,nbEnfantsAcharge,situationFamiliale,2emevoiture,categorie_label,categorie
0,49,F,914,1,En Couple,False,2,Familliale
1,18,M,563,4,En Couple,False,1,Haut de gamme
2,82,M,417,0,Célibataire,False,3,Economique
3,72,M,442,4,En Couple,False,1,Haut de gamme
4,41,M,592,0,Célibataire,False,4,Moyen
5,54,M,438,0,En Couple,False,4,Moyen
6,30,F,736,1,En Couple,False,2,Familliale
7,56,M,1325,2,En Couple,False,1,Haut de gamme
8,73,M,532,0,Célibataire,False,3,Economique
9,31,M,206,0,En Couple,False,3,Economique


## Enregistrenement du DataFrame avec Pickle

In [33]:
# Enregistrement du DataFrame
with open('data_processed/fusion_clients_immatriculations2.pkl', 'wb') as f1:
    pickle.dump(clients_immatriculations, f1)